# Twitter Analysis of @borussia account

## Getting Data

In [2]:
import tweepy
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

Loading variables from Twitter developer account (dont want to show them to the public)

In [3]:
consumer_key='fk1WXQHHuOKtDW08S8koKHJUx'
consumer_secret='YixJ1JSVmUW3eerSCF3Xk5i44LHjWEqaomW16wc3e68zyKAl2X'
access_token='910844887154733056-at915GYCu1CUdo9fmyx2le2LAgMgG4e'
access_token_secret='SCL4zPt2f3y7YaVUCfIYo8G31IBP0PtcMFKf3gteij9B1'

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


Creating lists to store the information from twitter account

In [5]:
number_of_tweets = 3000
tweets = []
likes = []
time = []
source = []
coordinates = []
retweets = []


for i in tweepy.Cursor(api.user_timeline, screen_name='borussia', tweet_mode = "extended").items(number_of_tweets):
    tweets.append(i.full_text)
    likes.append(i.favorite_count)
    time.append(i.created_at)
    source.append(i.source)
    coordinates.append(i.coordinates)
    retweets.append(i.retweet_count)

Create a dataframe with the extracted variables

In [6]:
df = pd.DataFrame({'tweets':tweets, 'likes':likes, 'time':time, 'source':source, 'coordinates':coordinates, 'retweets':retweets})

## Analyze and Clean Data

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   tweets       3000 non-null   object             
 1   likes        3000 non-null   int64              
 2   time         3000 non-null   datetime64[ns, UTC]
 3   source       3000 non-null   object             
 4   coordinates  0 non-null      object             
 5   retweets     3000 non-null   int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(3)
memory usage: 140.8+ KB


In [8]:
# Checking the numerical variables
df.describe()

,likes,retweets
count,3000.000000,3000.000000
mean,168.351333,7.251667
std,441.954745,40.673641
min,0.000000,0.000000
25%,46.000000,1.000000
50%,89.000000,3.000000
75%,178.000000,5.000000
max,12133.000000,1711.000000


In [9]:
# How many different value are there in the column 'coordinates'? Should I drop this column?
df['coordinates'].unique()


array([None], dtype=object)

In [10]:
df = df.drop('coordinates', axis=1)


In [11]:
# Drop all retweets from column tweets and reset the index

df = df[~df.tweets.str.contains('RT')]
df = df.reset_index(drop=True)
display(df)

# 2925 rows remainaing, there were only 75 retweets. Now we need to reset the index. Lets start with analyzing the data!


,tweets,likes,time,source,retweets
0,Einfach nur 🫶\n\nDanke für euren unfassbaren S...,253,2022-07-10 16:44:29+00:00,Twitter for iPhone,4
1,🍿 Die Highlights vom Spiel 🆚 1860 München sind...,94,2022-07-10 16:23:29+00:00,Twitter for iPhone,4
2,"Zweiter Test, zweiter Sieg. 🤙\n\n#DieFohlen #B...",275,2022-07-10 15:13:03+00:00,Twitter for iPhone,13
3,Feierabend! #DieFohlen gewinnen zum Abschluss ...,393,2022-07-10 14:16:58+00:00,Twitter Web App,10
4,70‘: Doppelpack Flaco! 6:0.\n\n#BMGM60 #BMGamT...,129,2022-07-10 13:40:15+00:00,Twitter for iPhone,4
...,...,...,...,...,...
2920,Hütter: @jjscally_10 hat heute wieder mit uns ...,7,2021-11-18 12:06:35+00:00,Twitter Web App,0
2921,"Hütter: Bei @bensebainiRams hoffen wir, dass e...",8,2021-11-18 12:06:04+00:00,Twitter Web App,0
2922,Sportdirektor Max Eberl und Cheftrainer Adi Hü...,6,2021-11-18 12:01:53+00:00,Twitter Web App,0
2923,🎙 Spieltags-PK mit Max Eberl &amp; Adi Hütter\...,50,2021-11-18 11:31:10+00:00,Twitter Web App,2


In [12]:
# add a cloumn containing the length of tweets

df['tweet_length'] = df['tweets'].apply(len)

# add a cloumn differiantiate between citations (1) and non-citations (0)

df['zitat'] = df.tweets.str.contains(': ')
df['zitat'] = df.zitat.astype('int')

Save as csv file

In [13]:
df.to_csv('twitter_bgm.csv', index=False)

In [14]:
df = pd.read_csv('twitter_bgm.csv')

In [15]:
# check correlations

df.corr()

,likes,retweets,tweet_length,zitat
likes,1.000000,0.879579,-0.053324,-0.148230
retweets,0.879579,1.000000,-0.003160,-0.065737
tweet_length,-0.053324,-0.003160,1.000000,0.224881
zitat,-0.148230,-0.065737,0.224881,1.000000


High correlation between likes and retweets. Almost no correlation between tweet-length and likes/retweets

In [16]:
# Checking for correlations concerning source used (caterogirial variable) and likes received

from scipy import stats
 
F, p = stats.f_oneway(df[df.source=='Twitter Web App'].likes,
                      df[df.source=='Twitter for iPhone'].likes,
                     df[df.source=='TweetDeck'].likes,
                     df[df.source=='Twitter for Android'].likes,
                     df[df.source=='Twitter for Advertisers'].likes)
F, p

(0.19271909808766322, 0.9422872981076696)

There seems to be no correlation between source used and like received

In [17]:
df['zitat'] = df.tweets.str.contains(': ')
df['zitat'] = df.zitat.astype('int')
df.corr()

,likes,retweets,tweet_length,zitat
likes,1.000000,0.879579,-0.053324,-0.148230
retweets,0.879579,1.000000,-0.003160,-0.065737
tweet_length,-0.053324,-0.003160,1.000000,0.224881
zitat,-0.148230,-0.065737,0.224881,1.000000


A small negative correlation between citations and likes received (-0.15)

## Analyzing Top 100 and Flop 100 Tweets

In [18]:
top100_likes = df.sort_values('likes',ascending = False).iloc[:100,:]
flop100_likes = df.sort_values('likes',ascending = True).iloc[:100,:]

In [136]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

C:\Users\gmult\AppData\Local\Temp\ipykernel_9960\3462665596.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


Create WordCloud for top100_likes and flop100_likes

In [19]:
text100 = top100_likes['tweets']
text100.to_csv('top100.csv', index=False)

flop100 = flop100_likes['tweets']
flop100.to_csv('flop100.csv', index=False)

In [20]:
top_text = open('top100.csv', mode='r', encoding='utf-8').read()
flop_text = open('flop100.csv', mode='r', encoding='utf-8').read()


In [21]:
# List notwords include fill word in german, that are used widely without much information for the wordcloud (filling words).

notwords = "mit nicht https ein nach einen wird uns wieder zu gut wie sehr alle sich sie wie es noch haben auch aber eine viele sind er werden aber dann aus sein t co de zum dass deine dem und in für bei hat da von Der das den wir ist die auf im Borussia"
list_notwords = notwords.split()
STOPWORDS.update(list_notwords)

In [24]:
x, y = np.ogrid[:1000, :1000]

mask = (x - 500) ** 2 + (y - 500) ** 2 > 400 ** 2
mask = 255 * mask.astype(int)

wc = WordCloud(
                background_color="white",
                stopwords=STOPWORDS,
                width=600, 
                height=400, 
                mask=mask).generate(top_text)

wc.to_file('top100_wordcloud.png')

wclast = WordCloud(
                background_color="grey",
                stopwords=STOPWORDS,
                width=600, 
                height=400, 
                mask=mask).generate(flop_text)

wclast.to_file('flop100_wordcloud.png')